AMAZON FINE FOOD REVIEWS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import nltk

In [2]:
import string
import seaborn as sn


In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [4]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
import re
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [6]:
con = sqlite3.connect(r'C:\Users\shwet\python projects\projects\database.sqlite')

In [7]:
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""", con)

In [8]:
print(filtered_data.head(5))

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [9]:
def partition(x):
    if x < 3:
        return 0
    return 1

In [10]:
ActualScore = filtered_data['Score']
PositiveNegative = ActualScore.map(partition)
filtered_data['Score'] = PositiveNegative

In [11]:
print(filtered_data.head(5))
print(filtered_data.shape)

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      1  1303862400   
1                     0                       0      0  1346976000   
2                     1                       1      1  1219017600   
3                     3                       3      0  1307923200   
4                     0                       0      1  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [12]:
display = pd.read_sql_query(""" SELECT ProductId, UserId, ProfileName, Score, Time, COUNT(*) FROM Reviews GROUP BY UserId
HAVING COUNT(*)>2""" , con)

In [13]:
print(display.head(5))
display.shape

    ProductId              UserId             ProfileName  Score        Time  \
0  B005HG9ESG  #oc-R11D9D7SHXIJB9  Louis E. Emory "hoppy"      5  1342396800   
1  B005HG9ESG  #oc-R11O5J5ZVQE25C           Penguin Chick      5  1346889600   
2  B005HG9ESG  #oc-R12MGTQS5KZZRV       SKY2110 "SKY2110"      5  1344211200   
3  B005HG9ESG  #oc-R13NNUL4EKL4FL        N. Chernyavskaya      1  1348358400   
4  B005HG9ESG  #oc-R14ZSRYW2YB41B              A. Crafton      5  1346284800   

   COUNT(*)  
0         3  
1         3  
2         3  
3         3  
4         3  


(48010, 6)

In [14]:
display['COUNT(*)'].sum()

327747

In [15]:
display = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 0 AND UserId = "AR5J8UI46CURR" ORDER BY ProductId""", con)

In [16]:
print(display)
display.shape

       Id   ProductId         UserId      ProfileName  HelpfulnessNumerator  \
0   78445  B000HDL1RQ  AR5J8UI46CURR  Geetha Krishnan                     2   
1  138317  B000HDOPYC  AR5J8UI46CURR  Geetha Krishnan                     2   
2  138277  B000HDOPYM  AR5J8UI46CURR  Geetha Krishnan                     2   
3   73791  B000HDOPZG  AR5J8UI46CURR  Geetha Krishnan                     2   
4  155049  B000PAQ75C  AR5J8UI46CURR  Geetha Krishnan                     2   

   HelpfulnessDenominator  Score        Time  \
0                       2      5  1199577600   
1                       2      5  1199577600   
2                       2      5  1199577600   
3                       2      5  1199577600   
4                       2      5  1199577600   

                             Summary  \
0  LOACKER QUADRATINI VANILLA WAFERS   
1  LOACKER QUADRATINI VANILLA WAFERS   
2  LOACKER QUADRATINI VANILLA WAFERS   
3  LOACKER QUADRATINI VANILLA WAFERS   
4  LOACKER QUADRATINI VANILLA WAFERS

(5, 10)

In [17]:
final = filtered_data.drop_duplicates(subset={'UserId', 'ProfileName','Score', 'Time', 'Text'}, keep='first', inplace=False)

In [18]:
final.shape

(4986, 10)

In [19]:
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.72

In [21]:
print(final.head(5))

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      1  1303862400   
1                     0                       0      0  1346976000   
2                     1                       1      1  1219017600   
3                     3                       3      0  1307923200   
4                     0                       0      1  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [22]:
display = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=0 AND Id=44737 OR Id=64422 ORDER BY ProductId""" , con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [24]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final.shape

(4986, 10)

In [26]:
final['Score'].value_counts()

1    4178
0     808
Name: Score, dtype: int64

In [27]:
final['Text'].head()

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: Text, dtype: object

In [29]:
count_vect = CountVectorizer()
final_count = count_vect.fit_transform(final['Text'].values)

In [34]:
final_count.get_shape()

(4986, 13510)